To create a predictive model for the parts that doesn't have sensors...

In [1]:
import pandas as pd
import numpy as np
import gmaps as gmaps
import json 
import datetime 
#from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv("C:/Users/audre/OneDrive/Bureau/res.csv", sep=",")
##remove the row when an information is missing
nan = float("NaN")
df.replace("", nan, inplace=True)
df.dropna(inplace=True)

In [3]:
##change the format of the date
df['attributes.date_debut'] = [datetime.datetime.fromtimestamp(i /1000) for i in df['attributes.date_debut']]
df['attributes.date_fin'] = [datetime.datetime.fromtimestamp(i /1000) for i in df['attributes.date_fin']]
df = df.reset_index() ##reset the index of the dataframe


In [4]:
year = int(input("Year: "))
month = int(input("Month: "))
day = int(input("Day: "))
hour = int(input ("Hour: "))
date = datetime.datetime(year,month,day,hour)

Year: 2021
Month: 02
Day: 02
Hour: 14


In [5]:
df=df.loc[df['attributes.valeur']>=0] #Only positive values
df.reset_index(drop=True, inplace=True) #Reset index after removing the outliers
df

,index,attributes.nom_dept,attributes.code_station,attributes.nom_station,attributes.nom_com,attributes.date_debut,attributes.date_fin,attributes.nom_poll,attributes.valeur,geometry.x,geometry.y
0,0,HAUTE-GARONNE,FR50021,Toulouse-Mazades Urbain,TOULOUSE,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,19.7,1.438610,43.6236
1,1,HAUTE-GARONNE,FR50030,Toulouse-Berthelot Urbain,TOULOUSE,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,20.5,1.444030,43.5873
2,2,HAUTE-GARONNE,FR50060,Montauban - Ramierou Urbain,MURET,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,41.3,1.375310,44.0129
3,3,HAUTES-PYRENEES,FR50052,Tarbes-Dupuy Urbain,TARBES,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,15.7,0.062222,43.2303
4,4,AVEYRON,FR50815,Rodez - Camonil Urbain,RODEZ,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,14.4,2.570700,44.3570
...,...,...,...,...,...,...,...,...,...,...,...
349586,360912,HAUTE-GARONNE,FR50040,Toulouse-SETMI Chapitre Industriel,TOULOUSE,2021-03-30 12:00:00,2021-03-30 13:00:00,PM10,22.9,1.403920,43.5564
349587,360913,HAUTES-PYRENEES,FR50042,Lourdes-Lapaca Urbain,LOURDES,2021-03-30 12:00:00,2021-03-30 13:00:00,PM10,40.2,-0.038889,43.0969
349588,360914,AVEYRON,FR50815,Rodez - Camonil Urbain,RODEZ,2021-03-30 14:00:00,2021-03-30 15:00:00,PM10,38.8,2.570700,44.3570
349589,360915,TARN,FR50053,Castres-Travet Urbain,CASTRES,2021-03-30 12:00:00,2021-03-30 13:00:00,PM10,24.5,2.230000,43.6106


In [6]:
!pip install folium

In [7]:
##Select the department you want to see on the heatmap
from IPython.html import widgets
from IPython.display import display

departments = {"Ariège", "Aude", "Aveyron", "Gard", "Haute-Garonne", "Gers", "Hérault", "Lot", "Lozère", "Hautes-Pyrénées", "Pyrénées-Orientales", "Tarn", "Tarn-et-Garonne"}

selectDepartment = widgets.Dropdown(options=departments)
display(selectDepartment)

C:\Users\audre\anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Dropdown(options=('Aveyron', 'Lozère', 'Haute-Garonne', 'Tarn', 'Tarn-et-Garonne', 'Hérault', 'Lot', 'Aude', '…

In [8]:
##Let's keep the values only in Montpellier. To do so, i draw a rectangle aroung the city and recovered the coordinates


dfMontpellier = df
frameOfMontpellier = dfMontpellier.loc[(df['geometry.x'] >= 3.6) & (dfMontpellier['geometry.x'] <= 4.0) & (dfMontpellier['geometry.y'] >= 43.5) & (dfMontpellier['geometry.y'] <= 43.7)]
frameOfMontpellier.reset_index()

,level_0,index,attributes.nom_dept,attributes.code_station,attributes.nom_station,attributes.nom_com,attributes.date_debut,attributes.date_fin,attributes.nom_poll,attributes.valeur,geometry.x,geometry.y
0,36,36,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 18:00:00,2020-11-12 19:00:00,PM10,29.5,3.88689,43.5915
1,62,62,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 18:00:00,2020-11-12 19:00:00,PM2.5,10.6,3.88689,43.5915
2,83,83,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 19:00:00,2020-11-12 20:00:00,PM10,29.2,3.88689,43.5915
3,98,98,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 19:00:00,2020-11-12 20:00:00,PM2.5,15.9,3.88689,43.5915
4,116,116,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2020-11-12 20:00:00,2020-11-12 21:00:00,PM2.5,16.4,3.88689,43.5915
...,...,...,...,...,...,...,...,...,...,...,...,...
15597,349478,360781,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2021-03-30 11:00:00,2021-03-30 12:00:00,PM10,31.5,3.88689,43.5915
15598,349553,360879,HERAULT,FR50200,Montpellier - Prés d Arènes Urbain,MONTPELLIER,2021-03-30 11:00:00,2021-03-30 12:00:00,PM2.5,7.9,3.88689,43.5915
15599,349564,360890,HERAULT,FR50203,Montpellier - Pompignane Trafic,MONTPELLIER,2021-03-30 11:00:00,2021-03-30 12:00:00,PM2.5,21.9,3.89861,43.6100
15600,349567,360893,HERAULT,FR50203,Montpellier - Pompignane Trafic,MONTPELLIER,2021-03-30 12:00:00,2021-03-30 13:00:00,PM10,27.2,3.89861,43.6100


In [9]:
##Calculation of the index to know the quality of the air and then print it with markers on the Folium map

##Gives the mean for each type for particule (PM10 and PM2.5) per day
dfParticuleTypePerDay = frameOfMontpellier.set_index('attributes.date_debut').groupby(['attributes.nom_poll', pd.Grouper(freq='d')]).mean()

##Let's define a function that return the color of the marker according to the ATMO Air Quality calculation
##Values are only relevant for daily mean according to ATMO

def quality(value, PMType):
    if PMType == "PM2.5":
        if value <10:
            print('blue')
        elif 11<value<20:
            print('green')
        elif 21<value<25:
            print('yellow')
        elif 26<value<50:
            print('orange')
        elif 51<value<75:
            print('red')
        elif value>75:
            print('purple')
    else:
        if value <20:
            print('blue')
        elif 21<value<40:
            print('green')
        elif 41<value<50:
            print('yellow')
        elif 51<value<100:
            print('orange')
        elif 101<value<150:
            print('red')
        elif value>150:
            print('purple')
        
#colo = quality(35, 'PM10')
#len(dfParticuleTypePerDay.columns)
dfParticuleTypePerDay

index  attributes.valeur  \
attributes.nom_poll attributes.date_debut                                     
PM10                2020-10-16               3263.250000           4.500000   
                    2020-10-17               3808.583333           6.487500   
                    2020-10-18               4939.325581          10.834884   
                    2020-10-19               6101.043478          19.615217   
                    2020-10-20               7009.812500          22.393750   
...                                                  ...                ...   
PM2.5               2021-03-26             334250.363636          22.384091   
                    2021-03-27             337277.875000           4.320833   
                    2021-03-28             342807.586957           4.615217   
                    2021-03-29             351677.957447           7.674468   
                    2021-03-30             357259.458333           8.525000   

                                           geometry.x  geometry.y  
attributes.nom_poll attributes.date_debut                          
PM10                2020-10-16               3.892750   43.600750  
                    2020-10-17               3.892750   43.600750  
                    2020-10-18               3.892069   43.599674  
                    2020-10-19               3.892495   43.600348  
                    2020-10-20               3.892750   43.600750  
...                                               ...         ...  
PM2.5               2021-03-26               3.893016   43.601170  
                    2021-03-27               3.892750   43.600750  
                    2021-03-28               3.892750   43.600750  
                    2021-03-29               3.892875   43.600947  
                    2021-03-30               3.892750   43.600750  

[332 rows x 4 columns]

In [10]:
from IPython.html import widgets
from IPython.display import display

d = {"PM10" : "PM10", "PM2.5" : "PM2.5"}

Particule = widgets.Dropdown(options=d)
display(Particule)

Dropdown(options={'PM10': 'PM10', 'PM2.5': 'PM2.5'}, value='PM10')

In [11]:
date_time = date.strftime('20%y%m%d')
##Return mean value of Day D 
meanOfParticules = dfParticuleTypePerDay.loc[Particule, date_time]
#meanOfParticules
#color = quality(meanOfParticules[1], Particule)
Pollutant = meanOfParticules[1]



KeyError: Dropdown(options={'PM10': 'PM10', 'PM2.5': 'PM2.5'}, value='PM10')

In [12]:
##Using folium module
import folium 
from folium import plugins

#locationForMeanX = meanOfParticules[2]
#locationForMeanY = meanOfParticules[3]
locations = df.iloc[df.loc[(df["attributes.date_debut"] == date)].index,[10,9]]
#messageInPopup = "Amout of pollutant", Pollutant
maxZoom = 10

m=folium.Map(location = [43.610769, 3.876716], min_zoom = 0, zoom_control = True)
m.add_child(plugins.HeatMap(locations, radius=15, name='Heatmap of Montpellier', overlay=True, show = True))
#folium.Marker([locationForMeanY,locationForMeanX], popup=messageInPopup).add_to(m)
m

In [14]:
m.getZoom()

AttributeError: 'Map' object has no attribute 'getZoom'

In [ ]:
len(frameOfMontpellier.columns)

In [ ]:
locations